# --- Day 4: Passport Processing ---

## Imports

In [1]:
import tensorflow as tf

In [ ]:
byr (Birth Year)
cid (Country ID)
ecl (Eye Color)
eyr (Expiration Year)
hcl (Hair Color)
hgt (Height)
iyr (Issue Year)
pid (Passport ID)

## Part 1

In [98]:
def check_validity(arr, part2):     
    arr_decoded = tf.strings.unicode_decode(arr[:,0], 'UTF-8').to_tensor()-97
    i = tf.argsort((26*arr_decoded[:,0]) + arr_decoded[:,1])
    arr = tf.gather(arr, i)
    
    if tf.shape(arr)[0] < 7:
        return False

    if tf.shape(arr)[0] == 7 and arr[1][0] == 'cid' or arr[0][0] == 'cid':
        return False
    
    offset = 0
    
    if tf.shape(arr)[0] == 8:
        offset = 1
    
    if not part2:
        return True
    
    # Just because it needs to be initialized everywhere
    x = 0.0
    
    x = tf.strings.to_number(arr[0][1])
    byr_t = x >= 1920 and x <= 2002

    ecl_t = tf.strings.regex_full_match(arr[1+offset][1], '^(amb|blu|brn|gry|grn|hzl|oth)$')

    x = tf.strings.to_number(arr[2+offset][1])
    eyr_t = x >= 2020 and x <= 2030

    hcl_t = tf.strings.regex_full_match(arr[3+offset][1], '^#[0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f]$')

    hgt_t = False
    hgt_s_size = tf.strings.length(arr[4+offset][1])
    x = tf.strings.to_number(tf.strings.substr(arr[4+offset][1], 0, hgt_s_size-2))

    if tf.strings.substr(arr[4+offset][1], -2, 2) == "cm":
        hgt_t = x >= 150 and x <= 193
    elif tf.strings.substr(arr[4+offset][1], -2, 2) == "in":
        hgt_t = x >= 59 and x <= 76

    x = tf.strings.to_number(arr[5+offset][1])
    iyr_t = x >= 2002 and x <= 2020
        
    pid_t = tf.strings.length(arr[6+offset][1]) == 9
        
    return byr_t and ecl_t and eyr_t and hcl_t and hgt_t and iyr_t and pid_t


@tf.function()
def solve(fname, part2=False):
    s = tf.io.read_file(fname)
    s = tf.strings.split(s, "\n\n")
    s = tf.map_fn(lambda s: tf.strings.regex_replace(s, '\n', ' '), s)
    s = tf.strings.split(s, ' ')
    s = tf.strings.split(s, ':')

    
    bools = tf.map_fn(lambda x: check_validity(x.to_tensor(), part2), s, fn_output_signature=tf.TensorSpec(shape=(), dtype=tf.bool, name=None))
    
    return tf.math.count_nonzero(bools)
        
solve('day4.txt', True)

<tf.Tensor: shape=(), dtype=int64, numpy=179>

## Part 1

In [99]:
%%timeit
solve('day4.txt')

837 ms ± 12.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Part 2

In [100]:
%%timeit
solve('day4.txt', part2=True)

1.13 s ± 4.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [87]:
import datetime
# Set up logging.
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.profiler.experimental.start(logdir)
solve('day4.txt')
tf.profiler.experimental.stop()